In [3]:
import os
import numpy as np
import json
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage.draw import polygon2mask
from skimage.measure import regionprops
from scipy.stats import pearsonr

# Paths to image and mask folders; adjust with your own complete paths
IMAGE_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\files"
CELL_MASK_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\masks\cells"
NUCLEUS_MASK_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\masks\nuclei"
OUTPUT_FOLDER = r"C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc"
DEBUG_FOLDER = os.path.join(OUTPUT_FOLDER, "debug")

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(DEBUG_FOLDER, exist_ok=True)

pixel_to_micron = 0.03529  # Scaling factor

def compute_pearson(channel1, channel2):
    valid_mask = (channel1 > 0) & (channel2 > 0)
    if np.sum(valid_mask) > 10:
        return pearsonr(channel1[valid_mask].flatten(), channel2[valid_mask].flatten())[0]
    return np.nan

def polygon_to_mask(image_shape, polygon_coords):
    swapped_coords = [(y, x) for x, y in polygon_coords]
    return polygon2mask(image_shape, swapped_coords)

def load_mask(mask_path, image_shape):
    if not os.path.exists(mask_path):
        return None

    with open(mask_path, 'r') as f:
        mask_data = json.load(f)

    labeled_mask = np.zeros(image_shape, dtype=int)
    label_id = 1

    for feature in mask_data['features']:
        geometry = feature['geometry']
        if geometry['type'] == 'Polygon':
            coords = geometry['coordinates'][0]
            poly_mask = polygon_to_mask(image_shape, coords)
            labeled_mask[poly_mask] = label_id
            label_id += 1
        elif geometry['type'] == 'MultiPolygon':
            for coords in geometry['coordinates']:
                poly_mask = polygon_to_mask(image_shape, coords[0])
                labeled_mask[poly_mask] = label_id
                label_id += 1
    return labeled_mask

for image_file in os.listdir(IMAGE_FOLDER):
    if not image_file.endswith(".tif"):
        continue

    image_path = os.path.join(IMAGE_FOLDER, image_file)
    cell_mask_path = os.path.join(CELL_MASK_FOLDER, image_file.replace(".tif", ".json"))
    nucleus_mask_path = os.path.join(NUCLEUS_MASK_FOLDER, image_file.replace(".tif", ".json"))

    image = tiff.imread(image_path)
    cell_mask = load_mask(cell_mask_path, image.shape[1:])
    nucleus_mask = load_mask(nucleus_mask_path, image.shape[1:])

    if cell_mask is None:
        print(f"Skipping {image_file} - no cell mask found")
        continue

    final_masks = np.copy(cell_mask)

    if nucleus_mask is not None:
        for nucleus_region in regionprops(nucleus_mask):
            y, x = nucleus_region.coords.T
            cell_label = cell_mask[y[0], x[0]]  # Get the enclosing cell label

            if cell_label > 0:
                final_masks[y, x] = 0  # Subtract nucleus from cell

    # Create and save debug image
    plt.figure(figsize=(10, 10))
    plt.imshow(image[0], cmap='gray')  # Display base image (channel 0)
    plt.imshow(final_masks, cmap='nipy_spectral', alpha=0.5)  # Overlay processed cell areas minus nuclei
    plt.axis('off')
    debug_image_path = os.path.join(DEBUG_FOLDER, image_file.replace(".tif", "_overlay.tiff"))
    plt.savefig(debug_image_path, format='tiff', bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"Saved overlay image with labels minus nuclei: {debug_image_path}")

    channel1 = image[1]
    channel2 = image[2]

    results = []
    for region in regionprops(final_masks, intensity_image=channel1):
        roi_mask = final_masks == region.label
        pearson_corr = compute_pearson(channel1[roi_mask], channel2[roi_mask])
        mean_intensity_c1 = np.mean(channel1[roi_mask])
        mean_intensity_c2 = np.mean(channel2[roi_mask])
        sum_intensity_c1 = np.sum(channel1[roi_mask])
        sum_intensity_c2 = np.sum(channel2[roi_mask])
        area_pixels = region.area
        area_microns = area_pixels * (pixel_to_micron ** 2) if pixel_to_micron else np.nan

        results.append({
            'Cell_ID': region.label,
            'Pearson_Correlation': pearson_corr,
            'Mean_Intensity_Channel1': mean_intensity_c1,
            'Mean_Intensity_Channel2': mean_intensity_c2,
            'Sum_Intensity_Channel1': sum_intensity_c1,
            'Sum_Intensity_Channel2': sum_intensity_c2,
            'Area_Pixels': area_pixels,
            'Area_Square_Microns': area_microns
        })

    df = pd.DataFrame(results)
    df.to_csv(os.path.join(OUTPUT_FOLDER, image_file.replace(".tif", ".csv")), index=False)


Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_15_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_16_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_17_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_18_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_19_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug\KRASG12D_BRAF_AAAA_20_overlay.tiff
Saved overlay image with labels minus nuclei: C:\Users\picca\Documents\PhD data\coloc_tests\results_no_nuc\debug